In [1]:
import nltk
import random
import pickle

from nltk.classify.scikitlearn import SklearnClassifier
from sklearn.naive_bayes import MultinomialNB, GaussianNB, BernoulliNB
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC

from statistics import mode
from nltk.tokenize import word_tokenize

In [2]:
def unpickle_data(filename):
    file = open(filename, "rb")
    data = pickle.load(file)
    file.close()
    return data

def pickle_data(data, filename):
    file = open(filename, "wb")
    pickle.dump(data, file)
    file.close()

In [3]:
from nltk.classify import ClassifierI
from statistics import mode

class VoteClassifier(ClassifierI):
    def __init__(self, *classifiers): # don't forget *
        self._classifiers = classifiers
        
    def classify(self, features):
        votes = []
        for c in self._classifiers:
            v = c.classify(features)
            votes.append(v)
        return mode(votes) # pick the most popular one. pos or neg        

    # mode() has an issue with even number, not sure a work around, so using odd number classifier.
# no unique mode; found 2 equally common values

    def confidence(self, features):
        votes = []
        for c in self._classifiers:
            v = c.classify(features)
            votes.append(v)
        
        choice_votes = votes.count(mode(votes)) # the number of most popular one
        conf = choice_votes / len(votes)        # the ratio of the most popular one.
        
        return conf

In [4]:
classifier = unpickle_data("naivebayesian_classifier.pickle")
MNB_classifier = unpickle_data("MNB_classifier.pickle")
BernoulliNB_classifier= unpickle_data("BernoulliNB_classifier.pickle")
LinearSVC_classifier = unpickle_data("LinearSVC_classifier.pickle")
SVC_classifier = unpickle_data("SVC_classifier.pickle")
NuSVC_classifier = unpickle_data("NuSVC_classifier.pickle")
LogisticRegression_classifier = unpickle_data("LogisticRegression_classifier.pickle")
SGDClassifier_classifier = unpickle_data("SGDClassifier_classifier.pickle")
voted_classifier = unpickle_data("voted_classifier.pickle")

# training_set = unpickle_data("NLTK_training_set.pickle")
# testing_set = unpickle_data("NLTK_test_set.pickle")

C:\Users\iambj\AppData\Local\conda\conda\envs\nltk\lib\site-packages\sklearn\base.py:312: UserWarning: Trying to unpickle estimator MultinomialNB from version 0.18.1 when using version 0.19.0. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
C:\Users\iambj\AppData\Local\conda\conda\envs\nltk\lib\site-packages\sklearn\base.py:312: UserWarning: Trying to unpickle estimator LabelEncoder from version 0.18.1 when using version 0.19.0. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
C:\Users\iambj\AppData\Local\conda\conda\envs\nltk\lib\site-packages\sklearn\base.py:312: UserWarning: Trying to unpickle estimator DictVectorizer from version 0.18.1 when using version 0.19.0. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
C:\Users\iambj\AppData\Local\conda\conda\envs\nltk\lib\site-packages\sklearn\base.py:312: UserWarning: Trying to unpickle estimator BernoulliNB from ve

In [5]:
#documents = []
#all_words = []

#short_pos = open("nltk_positive.txt", "r").read()
#short_neg = open("nltk_negative.txt", "r").read()

#allowed_word_types = ["J"]

#for p in short_pos.split('\n'):
#    documents.append( (p, "pos") )
#    words = word_tokenize(p)
#    pos = nltk.pos_tag(words)
#    for w in pos:
#        if w[1][0] in allowed_word_types:
#            all_words.append(w[0].lower())

#for p in short_neg.split('\n'):
#    documents.append( (p, "neg") )
#    words = word_tokenize(p)
#    pos = nltk.pos_tag(words)
#    for w in pos:
#        if w[1][0] in allowed_word_types:
#            all_words.append(w[0].lower())

#pickle_data(documents, "documents.pickle")

In [6]:
#from nltk.corpus import stopwords
#import string

#stop_words = set(stopwords.words("english"))
#stop_words = stop_words.union(set(string.punctuation))

#filtered_words = []
#for w in all_words:
#    if w not in stop_words:
#        filtered_words.append(w)
        
#filtered_words_dist = nltk.FreqDist(filtered_words)

#word_features = filtered_words_dist.most_common(5000)
#word_features = [word_feature[0] for word_feature in word_features]

#pickle_data(word_features, "word_features")

In [7]:
word_features = unpickle_data("word_features")

In [8]:
def find_features(document):
    words = word_tokenize(document) # BJ: Why do we need to change this instead of set just using the same method?
    features = {}
    for w in word_features: # Why not enumerating words in the document? --> Then no way to find "false" words.
        features[w] = (w in words) # True if w is in words/the document.
        
    return features

In [9]:
def sentiment(text):
    features = find_features(text)
    return voted_classifier.classify(features), voted_classifier.confidence(features)

In [15]:
print(sentiment("This movie was awesom. The acting was great, plot was wonderful, and there were pythons.. so yea!"))
print(sentiment("This movie was awesom."))
print(sentiment("This movie was utter junk."))
print(sentiment("There are absoultely zero phython"))

('pos', 1.0)
('neg', 0.8)
('neg', 1.0)
('neg', 0.8)


# https://apps.twitter.com/app/14732665/keys

In [21]:
from tweepy import Stream
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener
import json
import time

#consumer key, consumer secret, access token, access secret.
ckey="fsdfasdfsafsffa"
csecret="asdfsadfsadfsadf"
atoken="asdf-aassdfs"
asecret="asdfsadfsdafsdafs"

class listener(StreamListener):

    def on_data(self, data):
        try: 
            all_data = json.loads(data)
            tweet = all_data["text"]
            sentiment_value, confidence = sentiment(tweet)        
            print(tweet, sentiment_value, confidence)
            time.sleep(0.3)

            if confidence * 100 >= 80: 
                output = open("twitter-out.txt", "a")
                output.write(sentiment_value)
                output.write("\n")
                output.close()

            return(True)
        except:
            return(True)        

    def on_error(self, status):
        print(status)

auth = OAuthHandler(ckey, csecret)
auth.set_access_token(atoken, asecret)

twitterStream = Stream(auth, listener())
twitterStream.filter(track=["trump"])

RT @amvetsupport: For all those trump supporters who say trump gives a crap about vets, here is a thought. Instead of $25 billion on the id… neg 0.8
https://t.co/Hb9mHRexLs neg 0.8
Good lord, let something distract them. The MSM is becoming an international embarrassment w all their #fakenews… https://t.co/8NqibvXPZU neg 1.0
RT @Slate: Trump’s immigration proposal is already dead: https://t.co/GBA88JQds7 https://t.co/tJumJk6UJI neg 1.0
@USELESS_SEQUELS @BradMossEsq @BrianSchoeneman As long as Trump’s base is with him , the GOP will support him.  Pol… https://t.co/SXt0swFgWr neg 1.0
Humberto Fontova - President Trump Wins 'Medal of Life' From Int'l Human Rights Group (But Fake News Media Blacks O… https://t.co/TM8Wcm4C9A neg 0.8
@Adriennebertuc1 @TwitterSupport Yes dummy. Donald Trump will leave and therefore all his followers will be gone as well. neg 0.6
RT @Patriot_Mom_17: Let us not forget, Soros is a Notsi do you which is really interesting that you can put those two words together

RT @TreyYingst: NEW: Trump campaign adviser Walid Phares has been interviewed by the special counsel multiple times over the past year, acc… neg 0.6
RT @BestCityTokyo: 📽️セール中‼️

https://t.co/Vro0DsK7yK

🈹
#iPhone #Spotify #Steam #Amazon #Tinder #iOS #Android #Apple #Google #YouTuber #Sta… neg 0.8
RT @riotwomennn: Democrats are excited to learn that @RepJoeKennedy will give the official response of @TheDemocrats to Trump's State of Co… pos 1.0
RT @cmclymer: Let me put this in perspective:

The DoD estimates that $8 million is the total annual cost for transgender service members'… neg 1.0
Why does this team take L’s so frequently? Is this fair? https://t.co/tDgFQgKjiT pos 0.8
@DevinNunes is such a dirty, dirty, dirty, traitorboy. He’ll need a bleach scrub &amp; a hydrogen peroxide hosing to ge… https://t.co/kcD41jWTpU neg 0.6
@Telegraph Trump is absolutely right, it was an onslaught of negative stories fueled by opinions and not the facts!… https://t.co/NTYawmMvcv pos 0.6
RT @RVAwonk: &

ProtocolError: ('Connection broken: IncompleteRead(0 bytes read)', IncompleteRead(0 bytes read))